# 第7章_モデルの評価


書籍に記載のサンプルプログラムのままですと、  
将来のscikit-learnのバージョンアップによってのwarning（警告）が出ますが  
プログラム的には影響はありません。  
本Notebookでは、warnings モジュールを用いて警告を非表示にしています。

In [25]:
# 警告を非表示に設定
import warnings
warnings.filterwarnings('ignore')

# ホールドアウト法

In [86]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

In [87]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# k- 分割交差検証

In [26]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
iris = load_iris()
model = LogisticRegression()
# 初期値は3分割
scores = cross_val_score(model, iris.data, iris.target)
print(scores)
# cvに指定の数値を入れると分割数を自由に設定できる
scores = cross_val_score(model, iris.data, iris.target, cv=5)
print(scores)

[0.96078431 0.92156863 0.95833333]
[1.         0.96666667 0.93333333 0.9        1.        ]


# 層化 k 分割交差検証

In [27]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
kfold = KFold(n_splits=3) # 分類はデフォルトでは層化交差検証だが、KFoldを使 うと回帰でデフォルトの交差検証になる。
iris = load_iris()
model = LogisticRegression()
# [0. 0. 0.]
print(cross_val_score(model, iris.data, iris.target,cv=kfold))
# 層化して分割する代わりに、データのシャッフルが可能
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
# [0.9 0.96 0.96]
print(cross_val_score(model, iris.data, iris.target, cv=kfold))

[0. 0. 0.]
[0.9  0.96 0.96]


# 1 つ抜き交差検証(leave-one-out)

In [28]:
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
iris = load_iris()
model = LogisticRegression()
lo = LeaveOneOut()
scores = cross_val_score(model, iris.data, iris.target, cv=lo)
print(len(scores))
print(scores.mean())

150
0.9533333333333334


# シャッフル分割交差検証

In [29]:
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
iris = load_iris()
model = LogisticRegression()

In [39]:
shuffle_split = ShuffleSplit(test_size=.5, train_size=0.5,n_splits=10)

scores = cross_val_score(model, iris.data, iris.target, cv=shuffle_split)
print(scores)

[0.86666667 0.94666667 0.94666667 0.97333333 0.96       0.85333333
 0.97333333 0.90666667 0.88       0.93333333]


# 交差検証を用いたグリッドサーチ

In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.datasets import load_iris

iris = load_iris()
# サーチするパラメータのグリッド
param_grid={'C':[0.001,0.01,0.1,1,10,100],
'gamma':[0.001,0.01,0.1,1,10,100]}
# モデルSVC、作成したグリッド、交差検証戦略は5分割層化交差検証
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
# テストデータと訓練データに分ける
X_train, X_test, y_train, y_test = train_test_split(
iris.data, iris.target, random_state=0)
# 訓練データを用いてモデル構築。この際、cv=5にしているので訓練データは内部で訓練デー タ ( 小 )と 検 証 デ ー タ に 分 か れ て 5 分 割 層 化 交 差 検 証 さ れ て い る
grid_search.fit(X_train,y_train)
grid_search.score(X_test,y_test)

0.9736842105263158

In [42]:
print(grid_search.score) # 各スコア
print(grid_search.best_params_) # 最適化したパラメータ

<bound method BaseSearchCV.score of GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)>
{'C': 100, 'gamma': 0.01}


# 混同行列

In [43]:
from sklearn.metrics import confusion_matrix
# データの読み込み
y_test = [0,0,0,0,0,1,1,1,1,1]
pred = [0,1,0,0,0,0,0,1,1,1]
confusion = confusion_matrix(y_test, pred)
print(confusion) # 混同行列を出力

[[4 1]
 [2 3]]


In [50]:
y_test = [0, 1, 0, 1, 1]
pred = [0, 1, 0, 0, 1]

# 適合率、再現率、F値

In [51]:
from sklearn.metrics import precision_score
print('Precision:', precision_score(y_test, pred))
from sklearn.metrics import recall_score
print('Recall:', recall_score(y_test, pred)) # 書籍ではy_predとなっておりますが、predです。
from sklearn.metrics import f1_score
print('F1 score:', f1_score(y_test,pred))

Precision: 1.0
Recall: 0.6666666666666666
F1 score: 0.8


# 回帰評価方法(RMSE)

In [53]:
y_true = [0, 2, 2, 1, 1, 3] # 実際の値
pred = [0, 1.7, 3.1, 3.1, 1.6, 2.7] # 予測した値

In [56]:
from sklearn.metrics import mean_squared_error
from math import sqrt
# rmse = sqrt(mean_squared_error(実際の値, 予測した値))
rmse = sqrt(mean_squared_error(y_true, pred))
print(rmse)

1.0132456102380443


# 回帰評価方法(決定係数)

p.293にて、以下のコードが記載されておりますが、  
<code>
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
model = LinearRegression
model.fit(X, y)
result = model.score(X, y)
</code>

正しくは次のセルのコードを実行ください。

In [77]:
y_true = [2, 3, 1, 2]
pred = [1.9, 2.8 , 0.8, 1.8]

In [80]:
from sklearn.metrics import r2_score
r2_score(y_true, pred)

0.9349999999999999